In [15]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [13]:
with open('Frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [14]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83, 26, 64, 47, 60, 57, 51, 60, 65, 66, 51, 55, 60, 19,  0,  0, 61, 64,
         5,  1, 66, 54, 51,  1, 33, 61, 50, 51, 64, 60,  1, 36, 64, 61, 59, 51,
        66, 54, 51, 67, 65,  0,  0, 48, 71,  1, 33, 47, 64, 71,  1, 42, 61, 58,
        58, 65, 66, 61, 60, 51, 49, 64, 47, 52, 66,  1,  3, 27, 61, 50, 69, 55,
        60,  4,  1, 38, 54, 51, 58, 58, 51, 71,  0,  0,  0,  1, 23, 35, 34, 39,
        25, 34, 39, 38,  0,  0,  1, 32, 51, 66])


In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)